In [1]:
!pip install torch-geometric -q
!pip install rdkit -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 14.0 MB/s eta 0:00:00


# Creating a GAT Model

Importing the required libraries

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch_geometric
from torch_geometric.datasets import MoleculeNet, QM9, PPI
from torch_geometric.data import DataLoader
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [23]:
dataset = PPI(root="./")

print(f'Dataset: {dataset}:')
print('======================')
print("Dataset type: ", type(dataset))
print(f'Number of graphs: {len(dataset)}')
print("Dataset sample: ", dataset[0])
print("Sample  nodes: ", dataset[0].num_nodes)
print("Sample  edges: ", dataset[0].num_edges)
print("Sample  edge_index: ", dataset[0].edge_index.shape)
print("Sample  X: ", dataset[0].x.shape)
print("Sample  y: ", dataset[0].y.shape)

Dataset: PPI(20):
Dataset type:  <class 'torch_geometric.datasets.ppi.PPI'>
Number of graphs: 20
Dataset sample:  Data(x=[1767, 50], edge_index=[2, 32318], y=[1767, 121])
Sample  nodes:  1767
Sample  edges:  32318
Sample  edge_index:  torch.Size([2, 32318])
Sample  X:  torch.Size([1767, 50])
Sample  y:  torch.Size([1767, 121])


Observing a sample of the data

In [24]:
sample = dataset[0]

print(sample.num_features)
print(sample.edge_index.shape)
print(sample.x.shape)
print(sample.y.shape)

50
torch.Size([2, 32318])
torch.Size([1767, 50])
torch.Size([1767, 121])


Creating a helper function to turn edge indices to adjacency matrices

In [25]:
from torch_geometric.utils import to_networkx

def get_adjacency_matrix(data):
    # Get the edge index from the data object
    edge_index = data.edge_index

    # Convert the edge index to a NetworkX graph
    G = to_networkx(data)

    # Create an empty adjacency matrix with appropriate size
    num_nodes = G.number_of_nodes()
    adj_matrix = torch.zeros(num_nodes, num_nodes)

    # Fill the adjacency matrix based on edges
    for source, target in G.edges():
        adj_matrix[source, target] = 1

    return adj_matrix




# GAT

In [58]:
class GATLayer(MessagePassing):
    def __init__(self, in_features, out_features, dropout, alpha, concat=True):
        super(GATLayer, self).__init__()
        self.dropout       = dropout        # drop prob = 0.6
        self.in_features   = in_features    #
        self.out_features  = out_features   #
        self.alpha         = alpha          # LeakyReLU with negative input slope, alpha = 0.2
        self.concat        = concat         # conacat = True for all layers except the output layer.


        # Xavier Initialization of Weights
        # Alternatively use weights_init to apply weights of choice
        self.W = nn.Parameter(torch.zeros(size=(in_features, out_features)))
        nn.init.xavier_uniform_(self.W.data, gain=1.414)

        self.a = nn.Parameter(torch.zeros(size=(2*out_features, 1)))
        nn.init.xavier_uniform_(self.a.data, gain=1.414)

        # LeakyReLU
        self.leakyrelu = nn.LeakyReLU(self.alpha)

    def forward(self, input, adj):
        # Linear Transformation
        h = torch.mm(input, self.W) # matrix multiplication
        N = h.size()[0]

        # Attention Mechanism
        a_input = torch.cat([h.repeat(1, N).view(N * N, -1), h.repeat(N, 1)], dim=1).view(N, -1, 2 * self.out_features)
        e = self.leakyrelu(torch.matmul(a_input, self.a).squeeze(2))

        # Masked Attention
        zero_vec  = -9e15*torch.ones_like(e)
        attention = torch.where(adj > 0, e, zero_vec)

        attention = F.softmax(attention, dim=1)
        attention = F.dropout(attention, self.dropout, training=self.training)
        h_prime   = torch.matmul(attention, h)

        if self.concat:
            return F.elu(h_prime)
        else:
            return h_prime

In [59]:
x = sample.x.float()
y = sample.y
edge_index = sample.edge_index

conv = GATLayer(in_features=dataset.num_features, out_features=dataset.num_classes, dropout=0.2, alpha=0.2, concat=True)
out = conv(x, get_adjacency_matrix(dataset[0]))

print(out.shape)

torch.Size([1767, 121])


# GCN Model

In [26]:
from torch_geometric.utils import add_self_loops, degree

class GCNConv(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super().__init__(aggr='add')  # "Add" aggregation (Step 5).
        self.lin = nn.Linear(in_channels, out_channels, bias=False)
        self.bias = nn.Parameter(torch.empty(out_channels))

        self.reset_parameters()

    def reset_parameters(self):
        self.lin.reset_parameters()
        self.bias.data.zero_()

    def forward(self, x, edge_index):
        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))
        x = self.lin(x)
        row, col = edge_index
        deg = degree(col, x.size(0), dtype=x.dtype)
        deg_inv_sqrt = deg.pow(-0.5)
        deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0
        norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]
        out = self.propagate(edge_index, x=x, norm=norm)
        out = out + self.bias

        return out

    def message(self, x_j, norm):
        return norm.view(-1, 1) * x_j

In [19]:
x = sample.x.float()
y = sample.y
edge_index = sample.edge_index

conv = GCNConv(in_channels=dataset.num_features, out_channels=64)
out = conv(x, edge_index)

print(out.shape)

torch.Size([19, 64])


In [32]:
class GCNModel(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GCNModel, self).__init__()
        self.conv1 = GCNConv(in_channels, 64)
        self.conv2 = GCNConv(64, 128)
        self.fc = nn.Linear(128, out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = self.fc(x)

        return F.log_softmax(x, dim=1)

In [28]:
# testing model

x = sample.x.float()
y = sample.y
edge_index = sample.edge_index

model = GCNModel(in_channels=dataset.num_features, out_channels=dataset.num_classes)
output = model(x, edge_index)

print(output.shape)

torch.Size([1, 1767, 121])


In [34]:
epochs = 10
lr = 1e-3

optimizer = torch.optim.Adam(model.parameters(), lr=lr)
criterion = torch.nn.MSELoss()

model = model.to(device)
criterion = criterion.to(device)

for epoch in range(epochs):
    for data in dataset:
        data = data.to(device)
        x, edge_index, y = data.x.float(), data.edge_index, data.y
        model.train()
        optimizer.zero_grad()
        out = model(x, edge_index)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()
    print(f'Epoch: {epoch+1:03d}, Loss: {loss:.4f}')

Epoch: 001, Loss: 69.3032
Epoch: 002, Loss: 69.3031
Epoch: 003, Loss: 69.3030
Epoch: 004, Loss: 69.3028
Epoch: 005, Loss: 69.3027
Epoch: 006, Loss: 69.3026
Epoch: 007, Loss: 69.3025
Epoch: 008, Loss: 69.3024
Epoch: 009, Loss: 69.3023
Epoch: 010, Loss: 69.3023
